In [1]:
!rm -f watson_streaming_pipelines.py*
!wget https://raw.githubusercontent.com/Yura32000/practicals/master/watson_streaming_pipelines.py

--2017-11-26 07:43:48--  https://raw.githubusercontent.com/Yura32000/practicals/master/watson_streaming_pipelines.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8016 (7.8K) [text/plain]
Saving to: ‘watson_streaming_pipelines.py’

100%[======================================>] 8,016       --.-K/s   in 0s      

2017-11-26 07:43:48 (23.8 MB/s) - ‘watson_streaming_pipelines.py’ saved [8016/8016]



In [2]:
import watson_streaming_pipelines as wstp
########################################################################################################################

In [3]:
# init() function will be called once on pipeline initialization
# @state a Python dictionary object for keeping state. The state object is passed to the process function
def init(state):

    cos_credentials = {
        'iam_url':'https://iam.stage1.ng.bluemix.net/oidc/token',
        'api_key':'xhjheSC7AhSLtvapSDnbyFn17uWUqW5ccAOuHhQxnnEY',
        'resource_instance_id':'crn:v1:staging:public:cloud-object-storage:global:a/68a66698d275aeb48097f868957ab2ed:bbb5aa36-5525-4000-b129-bcb780195098::',
        'url':'https://s3-api.us-geo.objectstorage.uat.service.networklayer.com',
        'endpoint':'https://s3.us-west.objectstorage.uat.softlayer.net'
    }

    state['outmsg'], state['model'] = get_model( cos_credentials, 'pyml', 'LDA_test_1.model.pickle')
    state['stoplist'] = setStopWordList()

In [4]:
def get_model( credentials, bucket_name, object_name):
    """
    Load the model using the appropriate method (based on how it was saved).
    In this example, the model was saved using 'pickle.dumps( self.model)'.
    """
    import pickle
    import gzip

    model = None
    outmsg = None
    
    try:
        pickledData = wstp.get_from_cloud_object_storage( 
            api_key = cos_credentials['api_key'],
            full_object_path = bucket_name + '/' + object_name, 
            auth_endpoint = cos_credentials['iam_url'],
            service_endpoint = cos_credentials['endpoint']
        )
        outmsg += "len(pickledData) = " + str(len(pickedData)) + "; "

        # model = wstp.deserializeObject( pickedData)
        if pickledData.startswith(b"\x1f\x8b\x08"): # Magic signature for gzip
            model = pickle.loads( gzip.decompress( pickledData))
        else:
            model = pickle.loads( pickledData)

        if model and len(model) > 0:
            try:
                outmsg += "Read new model. Model topics = " + str(model.num_topics) + ", Dictionary tokens = " + str(len(model.id2word.keys()))                
            except:
                outmsg += "Error reading model; "
    except:
        outmsg = "Failed: get_from_cloud_object_storage(); "       

    return outmsg, model

In [5]:
def setStopWordList():
    import nltk

    stoplist = {}

    nltk.download("stopwords")
    stoplist = set(nltk.corpus.stopwords.words("english"))

    if len(stoplist) == 0: # Static, just in case
        stoplist = {'because', 'during', 'was', 'itself'}
        # stoplist = {'because', 'during', 'was', 'itself', 'should', 'by', 'haven', 'yourself', 'been', 're', 'ain', 'hadn', 'had', 'again', 'what', 'they', 'themselves', 'whom', 'you', 'all', 'both', 'on', 'isn', 'his', 'ourselves', 'that', 't', 'm', 'is', 'this', 'how', 'when', 'will', 'against', 'her', 'with', 'couldn', 'being', 'hasn', 'be', 'it', 'but', 'no', 'than', 'don', 'most', 'now', 'while', 'doesn', 'our', 'from', 'are', 'he', 'so', 'shouldn', 've', 'y', 'as', 'we', 'll', 's', 'himself', 'my', 'about', 'more', 'where', 'down', 'there', 'just', 'nor', 'theirs', 'such', 'who', 'to', 'before', 'him', 'me', 'has', 'o', 'its', 'were', 'did', 'can', 'same', 'then', 'have', 'few', 'aren', 'd', 'other', 'further', 'and', 'off', 'these', 'an', 'wasn', 'hers', 'your', 'weren', 'until', 'only', 'does', 'shan', 'i', 'own', 'not', 'or', 'myself', 'through', 'some', 'didn', 'at', 'out', 'why', 'needn', 'doing', 'above', 'after', 'wouldn', 'yourselves', 'very', 'having', 'herself', 'a', 'the', 'am', 'if', 'into', 'once', 'won', 'too', 'up', 'ours', 'here', 'those', 'each', 'in', 'over', 'ma', 'them', 'under', 'for', 'mustn', 'yours', 'mightn', 'below', 'between', 'which', 'do', 'any', 'she', 'of', 'their'}

    return stoplist

In [ ]:
state = {}
init(state)